In [ ]:
import tensorflow as tf

In [ ]:
!pip3 install tensorflow==1.15.3

  Using cached https://files.pythonhosted.org/packages/02/36/9a02e27f0ec248b676a380ffe910c1858e3af3027c0d4d513dd0b56a5613/tensorflow-1.15.3-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/de/62/2ee9cd74c9fa2fa450877847ba560b260f5d0fb70ee0595203082dafcc9d/tensorflow_estimator-1.15.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1e/e9/d3d747a97f7188f48aa5eda486907f3b345cd409f0a0850468ba867db246/tensorboard-1.15.0-py3-none-any.whl
  Found existing installation: tensorflow-estimator 2.0.1
    Uninstalling tensorflow-estimator-2.0.1:
      Successfully uninstalled tensorflow-estimator-2.0.1
  Found existing installation: tensorboard 2.0.2
    Uninstalling tensorboard-2.0.2:
      Successfully uninstalled tensorboard-2.0.2
  Found existing installation: tensorflow 2.0.0
    Uninstalling tensorflow-2.0.0:
      Successfully uninstalled tensorflow-2.0.0


In [ ]:
tf.__version__
# !protoc --version

'1.15.3'

In [ ]:
# Here's how to get at the hidden protocol buffers APIs in TensorFlow.
# See https://developers.google.com/protocol-buffers/docs/reference/python-generated
# for information on these generated classes.
# Generated Python module for tensorflow/core/framework/tensor.proto
tf.core.framework.tensor_pb2

In [ ]:
# Generated Python module for tensorflow/core/protobuf/saved_model.proto
tf.core.protobuf.saved_model_pb2

In [ ]:
# Contents of the generated Python module
dir(tf.core.protobuf.saved_model_pb2)

In [ ]:
# Download the Tensorflow model
# !wget 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz'
!tar -xzf ssd_mobilenet_v1_coco_2018_01_28.tar.gz
# !rm ssd_mobilenet_v1_coco_2018_01_28.tar.gz
# %cd ssd_mobilenet_v1_coco_2018_01_28/

In [ ]:
from tensorflow.core.framework import graph_pb2
from tensorflow.python.client import session
from tensorflow.python.platform import gfile
from tensorflow.python.framework import ops
from tensorflow.python.framework import importer

graph_def_proto = ""

# Import the protobuf graph definition from the frozen graph and load it in Tensorflow
# Reference: https://www.tensorflow.org/api_docs/python/tf/graph_util/import_graph_def
with session.Session(graph=ops.Graph()) as sess:
  with gfile.GFile("frozen_inference_graph.pb", "rb") as f:
    graph_def_proto = graph_pb2.GraphDef()
    graph_def_proto.ParseFromString(f.read())
    importer.import_graph_def(graph_def_proto)

In [ ]:
import numpy as np

# Interpret the python 'bytes' object as a buffer.   
def swap16(x):
  return np.frombuffer(x, dtype=np.uint16).byteswap().tobytes()

def swap32(x):
  return np.frombuffer(x, dtype=np.uint32).byteswap().tobytes()

def swap64(x):
  return np.frombuffer(x, dtype=np.uint64).byteswap().tobytes()

def byte_swap_array(content, bytes_per_elem):
    if bytes_per_elem == 1:
      return content

    elif bytes_per_elem == 2:
      return swap16(content)

    elif bytes_per_elem == 4:
      return swap32(content)

    elif bytes_per_elem == 8:
      return swap64(content)

    else:
      return content

In [ ]:
set_allowed = set([tf.string, tf.qint8, tf.quint8, tf.bool, tf.int8, tf.uint8])
set_16bits = set([tf.bfloat16, tf.half, tf.qint16, tf.quint16, tf.uint16, tf.int16])
set_32bits = set([tf.float32, tf.int32, tf.qint32, tf.uint32])
set_64bits = set([tf.int64, tf.uint64, tf.double])

# Separate the tensors according to tf.dtype of their tensor_content
def byte_swap_tensor(orig_tensor) -> bytes:

    datatype = orig_tensor.dtype
    content = orig_tensor.tensor_content
    bytes_per_elem = 0

    if datatype in set_allowed:
      bytes_per_elem = 1
    elif datatype in set_16bits:
      bytes_per_elem = 2
    elif datatype in set_32bits:
      bytes_per_elem = 4
    elif datatype in set_64bits:
      bytes_per_elem = 8
    elif datatype in set_complex64:
      bytes_per_elem = 4
    elif datatype in set_complex128:
      bytes_per_elem = 8
    else:
      print("Byteswapping not supported for datatype")
      return content
  
    return byte_swap_array(content, bytes_per_elem)

In [ ]:
# Iterate over the nodes of graphs in the top-level graph to identify all the tensors. The `tensor_content` field is 
# a memory dump having the same byte order of the system it was created on (x86 machine with little endian byte order).
# So, we convert the byte order of 'tensor_content' from little endian to big endian to make it usable on IBM Z.
node_names = []
for n in graph_def_proto.node:
    # print(n.name)
    for attr_name in n.attr:
        attr_value = n.attr[attr_name]
        if attr_value.HasField("tensor"):   
            tensor_value = attr_value.tensor
            if len(tensor_value.tensor_content) > 0:
              print(f"       Before: {attr_name} => {tensor_value.tensor_content}")
              swapped_tensor_content = byte_swap_tensor(tensor_value)
              tensor_value.tensor_content = swapped_tensor_content
              print(f"       After: {attr_name} => {tensor_value.tensor_content}")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



       After: value => b'\xbcOD\x1f\xbd\x04\xd5\xb3\xbb\xc7\xd8\xbd=\x84\x13\xb6\xbe\x03\x80i=y\xc1\xae\xbd\x03n\xf9\xbd\xb4kg\xbb\xa9\xceK\xbe\x14&\xd4\xbd\xe6\xe5\xbf=~\xad3=\x97\x8c\x11\xbcZ\x9d\x10\xbc\xad\xf4U\xbd2\xe7o\xbe\xc9\x06\xa2<r\xcb\xb7\xbc\x966\xd4<\xeaT\xfd=\xa0 M\xbd\x96\xde\x13=\x9f\x02|\xbe<\xdb\xf2\xbd\xcey\x08<\x8cg\xa2\xa5\xa1\xa5\\>\x170\xda\xbd\xa4[E=\x8c\x90>=\xdcK\xbd\xbd\xf1\x80y\xbd\xc3\x16V=\xce\xa5f<\xbb2(\xbd\x81\x95\xe7:DO\x8e\xbd\xda\xb9\xef=\x99\x95\xe4\xbb\xaa%\xa9\xbd\x84m\x1d\xbd\xca7\xbf\xbc\xd4\x01\xd2=(\x93r=\xa7i\xec=\x89`7\xbd\x97\xce\x8c\xbb\xa7>\x97=\x15\xb5\xfc\xbe\x08\xb91=\xb7\xd1\xf9>$\xfcI\xbd\x13?\xb9\xbe\x87\xbeO\xbdY\xe5\xb6\xbeL\x11\xcb>H\xcd:>\t,\xba\xbdx\x04q\xbd?\x10\x08\xbd\xddJ\x94>\xc6\xd7\xc2\xbd&"h\xbc\xd2\xa9\xbc=\n\xcd\n\xbe\x12\xec\xe0\xbb\xeb\xe6\x0f>w_\x93\xbd\xd4B%=\xdeu\xfc\xbd\x80%\xd8=\xf8\x18>\xbc\xc9\xc1*\xbd\xa6\x87\xa8<S\xad\xff\xbd\x0c1p=$\xca\xa1\xbe\x9c\x19=<\x05\x10\xf8\xbe8\x86|\xbc3\xe9\xea>\x02w\xf3<\x0c\x

In [ ]:
binary_saved_model_proto = graph_def_proto.SerializeToString()
len(binary_saved_model_proto)

29110773

In [ ]:
# Saving the rewitten data as a protobuf frozen graph
with open("swapped_frozen_inference_graph.pb", "wb") as f:
    f.write(binary_saved_model_proto)